In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [54]:
# lenders
USB= 504713         # US Bank
Wells=451965        #Wells Fargo
Quicken=7197000003  #Quicken Loan

loans_raw = pd.read_csv('AllData2017.csv')
#loans_df = loans_raw.loc[(loans_raw['respondent id'] == Wells) | (loans_raw['respondent id'] == USB) | (loans_raw['respondent id'] == Quicken), :]
#loans_df = loans_raw.loc[(loans_raw['respondent id'] == Wells), :]
#loans_df = loans_raw.loc[(loans_raw['respondent id'] == USB), :]
loans_df = loans_raw.loc[(loans_raw['respondent id'] == Quicken), :]
loans_df.head()

,Index,respondent id,agency code,loan type,property type,loan purpose,owner occ,loan amount,pre approval,action taken,...,denial reason 1,denial reason 2,denial reason 3,rate of spread,hoepa status,lien status,loan type modified,action modified,income cleaned,income loan ratio
38324,38325,7197000003,7,2,1,3,1,181,3,1,...,NaN,NaN,NaN,NaN,2,1,0,1,101,179
38325,38326,7197000003,7,1,1,3,1,422,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,238,177
38326,38327,7197000003,7,1,1,1,1,488,2,1,...,NaN,NaN,NaN,NaN,2,1,1,1,174,280
38327,38328,7197000003,7,2,1,3,1,178,3,1,...,NaN,NaN,NaN,NaN,2,1,0,1,54,330
38328,38329,7197000003,7,1,1,3,1,106,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,41,259


In [55]:
# Assign X (data) and y (target)
#drop all of the columns with non-numeric data
X=loans_df.drop(["action modified","respondent id", "state", "county", "loan type", "owner occ", "census tract", "hoepa status", "Index", "agency code", "income", "msa/md", "pre approval", "action taken", "denial reason 1", "denial reason 2", "rate of spread", "denial reason 3", "type of purchaser"],axis=1)

y=loans_df["action modified"]
print(X.shape, y.shape)
X.head()

(8381, 9) (8381,)


,property type,loan purpose,loan amount,applicant race 1,sex,lien status,loan type modified,income cleaned,income loan ratio
38324,1,3,181,6,3,1,0,101,179
38325,1,3,422,6,3,1,1,238,177
38326,1,1,488,6,3,1,1,174,280
38327,1,3,178,5,1,1,0,54,330
38328,1,3,106,5,2,1,1,41,259


In [56]:
X_continuous = pd.DataFrame(X, columns=['income cleaned', 'loan amount', 'income loan ratio'])

# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_continuous)


In [57]:
# Transform the training and testing data using the X_scaler

X_scaled = X_scaler.transform(X_continuous)

In [58]:
# One-hot encoding
property_type = to_categorical(X["property type"])
loan_purpose = to_categorical(X["loan purpose"])
race = to_categorical(X["applicant race 1"])
sex = to_categorical(X["sex"])
lien_status = to_categorical(X["lien status"])
loan_type = to_categorical(X["loan type modified"])


In [59]:
categorical_X = np.hstack([property_type, loan_purpose, race, sex, lien_status, loan_type])

print(categorical_X.shape)


(8381, 21)


In [60]:
all_X = np.hstack([categorical_X, X_scaled])

print(all_X[0])


[0.         1.         0.         0.         0.         1.
 0.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         1.         0.
 1.         1.         0.         0.07340116 0.15710254 0.20457143]


In [61]:
# Use train_test_split to create training and testing data

X_train, X_test, y_train, y_test = train_test_split(all_X, y)
#X_train.dtypes

In [62]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [63]:
# First, create a normal neural network with 
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=24))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [64]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [65]:
# Fit the model to the training data
model.fit(
    X_train,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
 - 0s - loss: 0.5781 - acc: 0.7300
Epoch 2/500
 - 0s - loss: 0.5747 - acc: 0.7306
Epoch 3/500
 - 0s - loss: 0.5744 - acc: 0.7306
Epoch 4/500
 - 0s - loss: 0.5717 - acc: 0.7306
Epoch 5/500
 - 0s - loss: 0.5724 - acc: 0.7306
Epoch 6/500
 - 0s - loss: 0.5706 - acc: 0.7306
Epoch 7/500
 - 0s - loss: 0.5711 - acc: 0.7306
Epoch 8/500
 - 0s - loss: 0.5707 - acc: 0.7309
Epoch 9/500
 - 0s - loss: 0.5704 - acc: 0.7313
Epoch 10/500
 - 0s - loss: 0.5700 - acc: 0.7313
Epoch 11/500
 - 0s - loss: 0.5694 - acc: 0.7313
Epoch 12/500
 - 0s - loss: 0.5707 - acc: 0.7313
Epoch 13/500
 - 0s - loss: 0.5680 - acc: 0.7316
Epoch 14/500
 - 0s - loss: 0.5686 - acc: 0.7309
Epoch 15/500
 - 0s - loss: 0.5673 - acc: 0.7316
Epoch 16/500
 - 0s - loss: 0.5663 - acc: 0.7314
Epoch 17/500
 - 0s - loss: 0.5668 - acc: 0.7324
Epoch 18/500
 - 0s - loss: 0.5651 - acc: 0.7332
Epoch 19/500
 - 0s - loss: 0.5656 - acc: 0.7325
Epoch 20/500
 - 0s - loss: 0.5646 - acc: 0.7333
Epoch 21/500
 - 0s - loss: 0.5657 - acc: 0.7333
E

Epoch 171/500
 - 0s - loss: 0.5188 - acc: 0.7542
Epoch 172/500
 - 0s - loss: 0.5198 - acc: 0.7537
Epoch 173/500
 - 0s - loss: 0.5211 - acc: 0.7519
Epoch 174/500
 - 0s - loss: 0.5196 - acc: 0.7529
Epoch 175/500
 - 0s - loss: 0.5189 - acc: 0.7515
Epoch 176/500
 - 0s - loss: 0.5213 - acc: 0.7543
Epoch 177/500
 - 0s - loss: 0.5180 - acc: 0.7524
Epoch 178/500
 - 0s - loss: 0.5181 - acc: 0.7535
Epoch 179/500
 - 0s - loss: 0.5168 - acc: 0.7537
Epoch 180/500
 - 0s - loss: 0.5172 - acc: 0.7539
Epoch 181/500
 - 0s - loss: 0.5150 - acc: 0.7583
Epoch 182/500
 - 0s - loss: 0.5178 - acc: 0.7532
Epoch 183/500
 - 0s - loss: 0.5162 - acc: 0.7548
Epoch 184/500
 - 0s - loss: 0.5156 - acc: 0.7540
Epoch 185/500
 - 0s - loss: 0.5166 - acc: 0.7534
Epoch 186/500
 - 0s - loss: 0.5154 - acc: 0.7558
Epoch 187/500
 - 0s - loss: 0.5139 - acc: 0.7561
Epoch 188/500
 - 0s - loss: 0.5138 - acc: 0.7580
Epoch 189/500
 - 0s - loss: 0.5183 - acc: 0.7521
Epoch 190/500
 - 0s - loss: 0.5180 - acc: 0.7545
Epoch 191/500
 - 0s 

Epoch 339/500
 - 0s - loss: 0.4766 - acc: 0.7707
Epoch 340/500
 - 0s - loss: 0.4775 - acc: 0.7704
Epoch 341/500
 - 0s - loss: 0.4775 - acc: 0.7733
Epoch 342/500
 - 0s - loss: 0.4862 - acc: 0.7706
Epoch 343/500
 - 0s - loss: 0.4758 - acc: 0.7707
Epoch 344/500
 - 0s - loss: 0.4730 - acc: 0.7752
Epoch 345/500
 - 0s - loss: 0.4778 - acc: 0.7726
Epoch 346/500
 - 0s - loss: 0.4756 - acc: 0.7734
Epoch 347/500
 - 0s - loss: 0.4777 - acc: 0.7699
Epoch 348/500
 - 0s - loss: 0.4736 - acc: 0.7718
Epoch 349/500
 - 0s - loss: 0.4761 - acc: 0.7720
Epoch 350/500
 - 0s - loss: 0.4765 - acc: 0.7709
Epoch 351/500
 - 0s - loss: 0.4754 - acc: 0.7730
Epoch 352/500
 - 0s - loss: 0.4736 - acc: 0.7722
Epoch 353/500
 - 0s - loss: 0.4738 - acc: 0.7720
Epoch 354/500
 - 0s - loss: 0.4810 - acc: 0.7674
Epoch 355/500
 - 0s - loss: 0.4751 - acc: 0.7718
Epoch 356/500
 - 0s - loss: 0.4746 - acc: 0.7720
Epoch 357/500
 - 0s - loss: 0.4722 - acc: 0.7733
Epoch 358/500
 - 0s - loss: 0.4698 - acc: 0.7755
Epoch 359/500
 - 0s 

In [66]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Neural Network - Loss: 0.9997616137257059, Accuracy: 0.700381679389313


In [67]:
#make a prediction and compare it to the actual labels
predictions = model.predict_classes(X_test)
predict_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

In [68]:
# Calculate confusion matrix
from sklearn.metrics import confusion_matrix
output = confusion_matrix(y_test, predictions)
output

array([[  71,  492],
       [ 136, 1397]], dtype=int64)

In [ ]:
output_df = pd.DataFrame(output)
output_df.to_csv("confusion_matrix.csv")